#Global cost of living

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,classification_report
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=[15,6]

In [3]:
df1 = pd.read_csv('/content/cost-of-living.csv')
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/cost-of-living.csv'

In [ ]:
df1=df1.drop('Unnamed: 0',axis=1)
df1.head()

In [ ]:
df2 = pd.read_csv('/content/cost-of-living_v2.csv')
df2.head()

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df = pd.concat([df1,df2],axis=0)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['city','country'],axis=1)
df.head()

In [ ]:
y = df['data_quality']
x = df.drop('data_quality',axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming x is a DataFrame with multiple columns

# Determine the number of rows and columns for your subplot grid
r = 5
c = 12  # Increase the number of columns

it = 1
for i in x.columns:
    plt.subplot(r, c, it)
    sns.distplot(x.loc[:, i], hist=False)
    plt.grid()
    it += 1

plt.tight_layout()
plt.show()

In [ ]:
x.shape

In [ ]:
for i in x.columns:
    x.loc[:,i].fillna(x.loc[:,i].median(),inplace=True)

In [ ]:
for i in x.columns:
    print("The skewness of",i,"is: ",x.loc[:,i].skew())

In [ ]:
np.sqrt(x['x53']).skew()

In [ ]:
np.sqrt(x['x52']).skew()

In [ ]:
pd.Series(pd.Series(stats.yeojohnson(x['x53']))[0]).skew()

In [ ]:
pd.Series(pd.Series(stats.yeojohnson(x['x53']))[0])

In [ ]:
for i in x.columns:
    print(i,"",x.loc[:,i].skew())

In [ ]:
x['x52'].value_counts()

In [ ]:
x.head()

#Model Building

In [ ]:
xc = sm.add_constant(x)
xc.head()

In [ ]:
x.boxplot()

In [ ]:
model = sm.Logit(y,xc).fit()
print(model.summary())

In [ ]:
# Remove the statistically insignificant variables

xc=sm.add_constant(x)
cols=list(xc.columns)
while len(cols)>1:
    xc=xc[cols]
    model=sm.Logit(y,xc).fit()
    p=model.pvalues
    pmax=max(p)
    pid=p.idxmax()           # index of series with highest value
    if pmax>0.05:
        cols.remove(pid)
        print(pid,pmax)
    else:
        break
cols

In [ ]:
xc = xc[['const','x1','x2','x3','x8','x16','x19','x20','x25','x28','x30','x32','x33','x35','x36','x37',
 'x40','x41','x42','x44','x46','x48','x50','x53','x54','x55']]

In [ ]:
model = sm.Logit(y,xc).fit()
print(model.summary())

In [ ]:
x=x[['x1','x2','x3','x8','x9','x16','x17','x18','x19','x20','x25','x26','x28','x30','x32','x33','x35','x36','x37','x38',
 'x40','x41','x42','x43','x44','x46','x48','x50','x53','x54','x55']]

#Building other models

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=42)
print("The shape of xtrain is: ",xtrain.shape)
print("The shape of xtest is: ",xtest.shape)
print("The shape of ytrain is: ",ytrain.shape)
print("The shape of ytest is: ",ytest.shape)

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()

model_list = [lr,dt,rf,knn,gnb]

train_acc = []
test_acc = []

for i in model_list:
    i_model = i.fit(xtrain,ytrain)
    ytrain_pred = i_model.predict(xtrain)
    ytest_pred = i_model.predict(xtest)
    train_acc.append(accuracy_score(ytrain,ytrain_pred))
    test_acc.append(accuracy_score(ytest,ytest_pred))

In [ ]:
train_acc

In [ ]:
test_acc

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()


model_dict = {'Model':model_list,'Train Score':train_acc,'Test Score':test_acc}
model_dict

In [ ]:
model_summary = pd.DataFrame({'Model':model_list,'Train Score':train_acc,'Test Score':test_acc})
model_summary

#Finding the Best Parameters

In [ ]:
dt = DecisionTreeClassifier()
params=[{'criterion':['gini','entropy'],
             'max_depth':range(2,7),
             'min_samples_leaf':range(2,10)
             }]

grd = GridSearchCV(estimator=dt,param_grid=params,cv=3)

grd_model = grd.fit(xtrain,ytrain)

grd_model.best_params_

In [ ]:
dt = DecisionTreeClassifier(criterion=grd_model.best_params_.get('criterion'),
                           max_depth=grd_model.best_params_.get('max_depth'),
                           min_samples_leaf=grd_model.best_params_.get('min_samples_leaf'))

dt_model = dt.fit(xtrain,ytrain)

ytrain_pred = dt_model.predict(xtrain)
ytest_pred = dt_model.predict(xtest)

In [ ]:
print("The accuracy score of regularized Decision tree is: ",accuracy_score(ytrain,ytrain_pred))
print("The accuracy score of regularized Decision tree is: ",accuracy_score(ytest,ytest_pred))

In [ ]:
rf = RandomForestClassifier()
params=[{'n_estimators':[100,150,200,250],
       'criterion':['gini','entropy'],
             'max_depth':range(2,7),
             'min_samples_leaf':range(2,10)
             }]
rnd = RandomizedSearchCV(estimator=rf,param_distributions=params,cv=3)
rnd_model = rnd.fit(xtrain,ytrain)
rnd_model.best_params_

In [ ]:
print("The accuracy score of regularized Random Forest is: ",accuracy_score(ytrain,ytrain_pred))
print("The accuracy score of regularized Random Forest is: ",accuracy_score(ytest,ytest_pred))

# To reduce overfitting

In [ ]:
train_score=[]
test_score=[]
for i in [lr,dt,rf,knn,gnb]:
    bg=BaggingClassifier(base_estimator=i,n_estimators=10,oob_score=True)
    bg_model=bg.fit(xtrain,ytrain)
    ypred_train=bg_model.predict(xtrain)
    ypred_test=bg_model.predict(xtest)
    train_score.append(accuracy_score(ytrain,ypred_train))
    test_score.append(accuracy_score(ytest,ypred_test))

In [ ]:
pd.DataFrame({'model':['Logistic Regression','Decision Tree','Random Forest','KNeighbors Classifier','Gaussian Naive-Bayes'],
              'train_score':train_score,'test_score':test_score})

In [ ]:
rf = RandomForestClassifier()
rf_model = rf.fit(xtrain,ytrain)
rf_model.feature_importances_

In [ ]:
importances = pd.DataFrame({'Features':xtrain.columns,'Feature Importances':rf_model.feature_importances_})
importances

In [ ]:
imp = importances.sort_values(by='Feature Importances',ascending=False)
imp

In [ ]:
sns.barplot(y='Features',x='Feature Importances',data=imp)
plt.show()

In [ ]:
print("thank you")